In [ ]:
import torch # This is the main PyTorch module. It provides multi-dimensional tensors and a variety of mathematical operations that can be performed on these tensors.
import torch.nn as nn  #This is a base class for all neural network modules in PyTorch.

def make_context_vector(context, word_to_ix): #context: This is a list of words representing the context of a target word. word_to_ix: This is a dictionary that maps words to their corresponding indices. Each word in the context needs to be converted to its index using this mapping.
    idxs = [word_to_ix[w] for w in context] #idxs: This is a list comprehension that iterates over each word in the context and retrieves its index from the word_to_ix dictionary.

    torch.tensor(idxs, dtype=torch.long):  #This creates a PyTorch tensor from the list of indices (idxs). The dtype=torch.long argument specifies that the tensor should have a data type of long (int64).
    return torch.tensor(idxs, dtype=torch.long)


CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text) #Create a set of unique words
vocab_size = len(vocab) # Calculate vocabulary size

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

# vocab: This likely represents the vocabulary, which is a set of unique words.

# enumerate(vocab): This function is used to iterate over the vocabulary, providing both the index (ix) and the word (word) for each element in the vocabulary.

# {word: ix for ix, word in enumerate(vocab)}: This dictionary comprehension creates a mapping (word_to_ix) from words to their corresponding indices in the vocabulary.

# {ix: word for ix, word in enumerate(vocab)}: Similarly, this dictionary comprehension creates a mapping (ix_to_word) from indices to their corresponding


data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

# raw_text: This likely represents a list or sequence of words, such as a piece of text.

# range(2, len(raw_text) - 2): This loop iterates over the indices of raw_text from 2 to len(raw_text) - 3. The purpose of excluding the first two and last two indices is to create a context window of size 4 around each word, ensuring that there are enough words on both sides for context.

# context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]: For each index i, this line creates a context window of size 4 around the word at index i. The context consists of the two words to the left and the two words to the right of the target word.

# target = raw_text[i]: This line extracts the target word at index i.

# data.append((context, target)): This appends a tuple to the data list, where the first element is the context window and the second element is the target word. This forms a dataset where each entry represents a context-target pair.


class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        # Embedding layer: Maps word indices to dense vectors
        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
         # Linear layer 1: Maps the sum of embedded context vectors to an intermediate vector
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        # Linear layer 2: Maps the intermediate vector to the log probabilities of each word in the vocabulary
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)


    def forward(self, inputs): #This represents the input tensor, typically containing indices of context words.
        embeds = sum(self.embeddings(inputs)).view(1,-1) #This applies the embedding layer to the input tensor, converting word indices into dense vectors.
        out = self.linear1(embeds) #This passes the reshaped embedding vector through the first linear layer.
        out = self.activation_function1(out) # This applies the ReLU activation function to the output of the first linear layer.
        out = self.linear2(out) #This passes the output of the ReLU activation through the second linear layer.
        out = self.activation_function2(out) #This applies the LogSoftmax activation function to the output of the second linear layer.
        return out #The final result (out) is the log probabilities of each word in the vocabulary.

    def get_word_emdedding(self, word): #This applies the embedding layer to the index, obtaining the word embedding./
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)#This reshapes the obtained word embedding into a 1-dimensional tensor with a size inferred to match the remaining dimensions.


model = CBOW(vocab_size, EMDEDDING_DIM) #creating an instance of the CBOW model with the specified vocabulary size (vocab_size) and embedding dimension (EMDEDDING_DIM).

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

#TRAINING
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix) #context_vector = make_context_vector(context, word_to_ix): This creates a context vector by converting the context words into indices using the word_to_ix mapping.

        log_probs = model(context_vector) #log_probs = model(context_vector): This passes the context vector through the CBOW model, obtaining log probabilities for each word in the vocabulary.

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]])) #This calculates the loss using a specified loss function (loss_function) by comparing the model's predictions (log_probs) with the target word's index. The loss is then added to the total loss for the current epoch.

    #optimize at the end of each epoch
    optimizer.zero_grad() #This line zeroes the gradients of all parameters before computing the gradients for the next mini-batch.
    total_loss.backward() #: This line computes the gradient of the loss with respect to the parameters of the model.
    optimizer.step() # This line updates the model parameters based on the computed gradients.

#TESTING
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs
